## Importing Libraries ##

In [1]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import timedelta
from datetime import datetime
import math

In [2]:
def filter_km(km_value):
    if abs(km_value) > 5:
        return 0
    return km_value

## Getting and cleaning the data ##

In [12]:
raw_data = pd.read_csv("MB1CWKHD2LPAL6794_input.csv")

In [13]:
# removing values which are blank, empty or zero

data = raw_data.loc[(raw_data['packet_sequence_id'] != np.nan) & \
                    (raw_data['packet_sequence_id'] != "") & \
                    (raw_data['packet_sequence_id'] != 0) ].copy()              

In [5]:
#data.sort_values(by ='obu_timestamp')

In [14]:
for column in data.columns:
    print(column)

Unnamed: 0
S_No
obu_id
obu_timestamp
vin
packet_sequence_id
p_frequency
latitude
longitude
altitude
heading
packet_status
gps_speed
no_of_satellites
location
gps_odometer
vehicle_type
frequency_from_w
longitudnal_acceleration
vehicle_state_info_1
gsm_signal_strength
vehicle_acceleration_sensor
ignition_status
vehicle_battery_potential
fuel_level
wheel_based_speed
engine_speed
fuel_consumption
actual_engine_percent_torque
engine_hours
engine_coolant_temperature
throttle_position
frictional_torque
engine_oil_pressure
clutch_pedal
brake_pedal
vehicle_distance
engine_oil_temperature
engine_intake_manifold_1_pressure
engine_intake_manifold_1_temperature
barometric_pressure
aft1_exhaust_gas_mass_flow_rate
drivers_demand_engine_percent_torque
vehicle_odometer
air_pressure_actual1
air_pressure_actual2
distance_to_empty
gsas_compliance
transmission_current_gear
ecu_mil_status
eng_operating_state
eng_current_oper_mode
eng_intake_air_mass_flow_rate
parking_brake_switch
engine_pto_set_switch
coola

In [15]:
data.head()

,Unnamed: 0,S_No,obu_id,obu_timestamp,vin,packet_sequence_id,p_frequency,latitude,longitude,altitude,...,transmission_mode_5_controller,transmission_mode_6_controller,transmission_mode_7_controller,transmission_mode_8_controller,aft1_diesel_particulate_filter_outlet_pressure,aft1_diesel_particulate_filter_outlet_temperature,aft1_diesel_exhaust_fluid_dosing_requested_quantity,combined_soot_load_gpl,reserved_1byte_param_1,transmission_mode_1_controller
0,0,1,2008102021,2022-12-5 04:46:41,MB1CWKHD2LPAL6794,6,NaN,26.016931,75.476896,353.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,2008102021,2022-12-5 06:26:25,MB1CWKHD2LPAL6794,52204,6.0,26.016898,75.476794,396.35,...,0.0,0.0,0.0,0.0,0.3,16.00,0.0,2.88,0.0,0.0
2,2,3,2008102021,2022-12-5 06:26:31,MB1CWKHD2LPAL6794,52205,6.0,26.016898,75.476794,396.35,...,0.0,0.0,0.0,0.0,0.3,16.00,0.0,2.88,0.0,0.0
3,3,4,2008102021,2022-12-5 06:26:37,MB1CWKHD2LPAL6794,52206,6.0,26.016898,75.476794,396.35,...,0.0,0.0,0.0,0.0,0.3,16.38,0.0,2.88,0.0,0.0
4,4,5,2008102021,2022-12-5 06:26:43,MB1CWKHD2LPAL6794,52207,6.0,26.016898,75.476794,396.35,...,0.0,0.0,0.0,0.0,0.3,18.09,0.0,2.88,0.0,0.0


In [16]:
data['distance_covered'] = data['vehicle_odometer'] - data.shift(1)['vehicle_odometer']


C:\Users\VijaySingh.Sisodia\AppData\Local\Temp\ipykernel_5980\3323794285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.replace(np.nan,0, inplace = True )


In [17]:
data['distance_covered'] = data.apply(lambda row: filter_km(row['distance_covered']), axis=1)

In [18]:
data = data.replace(np.nan,0)

In [19]:
data.distance_covered.sum()

647.4899999999907

## Distance Travelled calculation

**Distance Travelled as per Vehicle ODO**

In [20]:
#distance travelled as per vehicle odo
distance_travelled_vehicle_odo = np.max(data[data['vehicle_odometer'] != 0]['vehicle_odometer'])- \
np.min(data[data['vehicle_odometer'] != 0]['vehicle_odometer'])

In [21]:
print(f'Distance travelled as per vehicle odo is {distance_travelled_vehicle_odo:.1f} km')

Distance travelled as per vehicle odo is 647.7 km


**Distance travelled as per ECU ODO**

In [22]:
vehicle_ecu_distance_readings = data[data['vehicle_distance'] != 0]['vehicle_distance']

In [23]:
distance_travelled_ecu_odo = np.max(vehicle_ecu_distance_readings)-np.min(vehicle_ecu_distance_readings)

In [24]:
print(f'Distance travelled as per vehicle odo is {distance_travelled_ecu_odo:.1f} km')

Distance travelled as per vehicle odo is 623.9 km


## Fuel Consumption calculation ##

In [25]:
fuel_consumption_readings = data[data['fuel_consumption'] != 0]['fuel_consumption']

In [26]:
Fuel_consumption = np.max(fuel_consumption_readings)-np.min(fuel_consumption_readings)

In [27]:
print(f'Fuel consumption is {Fuel_consumption} lts')

Fuel consumption is 128.5 lts


## Running time calculations

**Active hours**

Total number of packets/600

In [28]:
total_active_hours = len(np.unique(data['packet_sequence_id']))/600

In [29]:
print(f'Total active hours are {total_active_hours:.2f} hours')

Total active hours are 18.52 hours


**Running Hours**

Total packets in below conditions / 600 <br>
- wheel base speed >=5 <br>
- engine speed >= 5

In [30]:
running_packets = data.loc[(data['wheel_based_speed']>=5) & (data['engine_speed']>=5)  ]['packet_sequence_id']

In [31]:
total_running_hours = len(np.unique(running_packets))/600

In [32]:
print(f'Total running hours are {total_running_hours:.2f} hours')

Total running hours are 15.33 hours


**Idling Hours** <br>
<br>
Total packets in below conditions / 600 <br>
- wheel base speed < 5 <br>
- engine speed <5


In [33]:
idling_packets = data.loc[(data['wheel_based_speed']<5) & (data['engine_speed']>5)  ]['packet_sequence_id'].copy()

In [34]:
total_idling_hours = len(np.unique(idling_packets))/600

In [35]:
print(f'Total idling hours are {total_idling_hours:.2f} hours')

Total idling hours are 1.89 hours


**Neutral driving hours**<br><br>
Total packets in below conditions / 600 <br>
- wheel base speed >= 5<br>
- engine speed > = 5<br>
- clutch pedal = 0<br>
- gear = 0



In [37]:
neutral_driving_packets = data.loc[(data['wheel_based_speed']>=5) & \
                                   (data['engine_speed']>=5)  & (data['clutch_pedal'] == 0) & \
                                   (data['transmission_current_gear'] == 0)]['packet_sequence_id'].copy()

In [38]:
total_neutral_driving_hours = len(np.unique(neutral_driving_packets))/600

In [39]:
print(f'Total neutral driving hours are {total_neutral_driving_hours:.2f} hours')

Total neutral driving hours are 0.89 hours


**Zero throttle running distance**
<br>
<br>
Total packets in below condition /600
- Throttle position = 0
- wheel based speed = 0

In [40]:
zero_throttle_packets = data.loc[(data['throttle_position']==0) & (data['wheel_based_speed']>5)  ]['packet_sequence_id'].copy()

In [41]:
zero_throttle_running_distance = len(np.unique(zero_throttle_packets))/600
print(f'Total zero throttle running distance is {zero_throttle_running_distance:.2f} km')

Total zero throttle running distance is 3.33 km


## Day time/Night time running calculation

**Daytime running distance** = Distance travelled between 5AM and 9PM

In [ ]:
def get_datetime(d_string):
    return 

In [49]:
data["obu_date"] = data.apply(lambda row, datetimerow['obu_timestamp']

In [50]:
first_day = data.iloc[0]['obu_date']
last_day = data.iloc[-1]['obu_date']
first_day_datetime = datetime.combine(first_day.date(), datetime.min.time())
last_day_datetime = datetime.combine(last_day.date(), datetime.max.time())

In [52]:
total_number_of_days = (last_day_datetime - first_day_datetime).days + 1

In [55]:
dates_list_5am = []
dates_list_9pm = []
for i in range(total_number_of_days):
    dates_list_5am.append(first_day_datetime + timedelta(days = i) + timedelta(hours = 5))
    dates_list_9pm.append(first_day_datetime + timedelta(days = i) + timedelta(hours = 21))

In [57]:
day_time_running_distance = 0

for i in range(len(dates_list_5am)):
    daytime_day = data.loc[(data['obu_date'] > dates_list_5am[i]) & (data['obu_date'] < dates_list_9pm[i])]
    max_odo = np.max(daytime_day[daytime_day['vehicle_odometer'] != 0]['vehicle_odometer'])
    min_odo = np.min(daytime_day[daytime_day['vehicle_odometer'] != 0]['vehicle_odometer'])
    #print(min_odo,max_odo)
    
    if(not (math.isnan(max_odo) and math.isnan(min_odo))):
        total_running_km = max_odo-min_odo
        day_time_running_distance += total_running_km   

        
night_time_running_distance = distance_travelled_vehicle_odo - day_time_running_distance

print(f'Total day time running distance is {day_time_running_distance:.2f} km')
print(f'Total night time running distance is {night_time_running_distance:.2f} km')
                     

Total day time running distance is 628.12 km
Total night time running distance is 19.62 km


## Miscelleneous Parameters

In [58]:
average_speed = distance_travelled_vehicle_odo / total_active_hours
halt_hours = total_number_of_days*24 - total_running_hours - total_idling_hours
running_percentage = (total_running_hours/total_active_hours)*100
idling_percentage = (total_idling_hours/total_active_hours)*100

print(f'Average speed of the vehicle is {average_speed:.2f} km/h ')
print(f'Total halt hours are {halt_hours:.2f} hrs')
print(f'Running % is {running_percentage:.2f} %')
print(f'Idling % is {idling_percentage:.2f} %')


Average speed of the vehicle is 34.98 km/h 
Total halt hours are 3678.77 hrs
Running % is 82.81 %
Idling % is 10.23 %


## Pivot for Vehicle Speed Vs Distance ##

In [59]:
groups= data.groupby([pd.cut(data.wheel_based_speed, np.arange(0, 110, 10), right = False)], dropna= False)
speed_vs_distance_covered_df = groups.distance_covered.sum().to_frame()
tot_distance = np.sum(speed_vs_distance_covered_df.distance_covered)

In [60]:
speed_vs_distance_covered_df['distance_travelled_percentage'] = speed_vs_distance_covered_df['distance_covered']/tot_distance

In [61]:
speed_vs_distance_covered_df

,distance_covered,distance_travelled_percentage
wheel_based_speed,,
"[0, 10)",7.21,0.011135
"[10, 20)",18.00,0.027800
"[20, 30)",31.94,0.049329
"[30, 40)",88.33,0.136419
"[40, 50)",295.86,0.456934
"[50, 60)",161.74,0.249795
"[60, 70)",33.03,0.051012
"[70, 80)",10.38,0.016031
"[80, 90)",1.00,0.001544


## Pivot for Throttle Vs Distance ##

In [62]:
data_throttle = data[data['engine_speed'] > 0]


groups= data_throttle.groupby([pd.cut(data.throttle_position, np.arange(-9, 120, 10))], dropna= False)
throttle_vs_distance_covered_df = groups.distance_covered.sum().to_frame()
tot_distance = np.sum(throttle_vs_distance_covered_df['distance_covered'])
throttle_vs_distance_covered_df['distance_travelled_percentage'] = throttle_vs_distance_covered_df['distance_covered']/tot_distance

In [63]:
throttle_vs_distance_covered_df

,distance_covered,distance_travelled_percentage
throttle_position,,
"(-9, 1]",146.66,0.226505
"(1, 11]",90.22,0.139338
"(11, 21]",100.18,0.154721
"(21, 31]",73.58,0.113639
"(31, 41]",51.13,0.078966
"(41, 51]",36.81,0.056850
"(51, 61]",28.50,0.044016
"(61, 71]",23.08,0.035645
"(71, 81]",34.55,0.053360


In [64]:
throttle_vs_distance_covered_df['distance_covered'].sum()

647.4899999999907

## Torque vs Distance Covered ##

In [65]:
groups= data_throttle.groupby([pd.cut(data.actual_engine_percent_torque, np.arange(-10, 120, 10))], dropna= False)
torque_vs_distance_covered_df = groups.distance_covered.sum().to_frame()
tot_distance = np.sum(torque_vs_distance_covered_df['distance_covered'])

In [66]:
torque_vs_distance_covered_df['distance_covered_percent'] = torque_vs_distance_covered_df['distance_covered']/tot_distance

In [67]:
torque_vs_distance_covered_df

,distance_covered,distance_covered_percent
actual_engine_percent_torque,,
"(-10, 0]",86.22,0.133160
"(0, 10]",72.50,0.111971
"(10, 20]",33.10,0.051120
"(20, 30]",72.72,0.112311
"(30, 40]",82.70,0.127724
"(40, 50]",100.51,0.155230
"(50, 60]",88.23,0.136265
"(60, 70]",34.86,0.053839
"(70, 80]",25.42,0.039259


## Engine Speed vs Distance Covered

In [68]:
groups= data.groupby([pd.cut(data.engine_speed, np.arange(0, 2600, 200), right = False)], dropna= False)
enginespeed_vs_distance_covered_df = groups.distance_covered.sum().to_frame()

In [69]:
tot_distance = np.sum(enginespeed_vs_distance_covered_df.distance_covered)
enginespeed_vs_distance_covered_df['distance_covered_percent'] = enginespeed_vs_distance_covered_df['distance_covered']/tot_distance

In [70]:
enginespeed_vs_distance_covered_df

,distance_covered,distance_covered_percent
engine_speed,,
"[0, 200)",0.00,0.000000
"[200, 400)",0.00,0.000000
"[400, 600)",1.61,0.002487
"[600, 800)",79.35,0.122550
"[800, 1000)",30.47,0.047059
"[1000, 1200)",146.64,0.226475
"[1200, 1400)",242.37,0.374322
"[1400, 1600)",121.43,0.187540
"[1600, 1800)",22.89,0.035352


## Gear vs Distance Covered

In [71]:
groups = data[data['wheel_based_speed']>5].groupby(by = 'transmission_current_gear', dropna = True)
gear_vs_distance_covered_df = groups.distance_covered.sum().to_frame()
gear_vs_distance_covered_df

,distance_covered
transmission_current_gear,
-1.0,0.12
0.0,65.34
1.0,0.00
2.0,2.37
3.0,10.51
4.0,26.65
5.0,40.52
6.0,500.73


In [72]:
tot_distance = np.sum(gear_vs_distance_covered_df['distance_covered'])
gear_vs_distance_covered_df['distance_covered_percent'] = gear_vs_distance_covered_df['distance_covered']/tot_distance

In [73]:
gear_vs_distance_covered_df

,distance_covered,distance_covered_percent
transmission_current_gear,,
-1.0,0.12,0.000186
0.0,65.34,0.101108
1.0,0.00,0.000000
2.0,2.37,0.003667
3.0,10.51,0.016263
4.0,26.65,0.041239
5.0,40.52,0.062701
6.0,500.73,0.774836


In [74]:
table_dict = {}

In [75]:
table_dict["Distance travelled(ECU Distance)"] = distance_travelled_ecu_odo
table_dict["Distance travelled (Vehicle odometer)"] = distance_travelled_vehicle_odo
table_dict["Fuel Consumption"]= Fuel_consumption
table_dict["KMPL ECU distance"] = Fuel_consumption/distance_travelled_ecu_odo
table_dict["KMPL Vehicle ODO"] = Fuel_consumption/distance_travelled_vehicle_odo
table_dict["Total hours"] = total_active_hours
table_dict["Total running hours"] = total_running_hours
table_dict["Total idling hours"] = total_idling_hours
table_dict["Halt time"] = halt_hours
table_dict["Running percentage"] = running_percentage
table_dict["Idling percentage"] = idling_percentage * 100
table_dict["Average Speed"] = average_speed

In [76]:
pd.DataFrame.from_dict(table_dict, orient ='index').to_excel("vitals.xlsx")

In [77]:
table_dict

{'Distance travelled(ECU Distance)': 623.8699999999953,
 'Distance travelled (Vehicle odometer)': 647.7399999999907,
 'Fuel Consumption': 128.5,
 'KMPL ECU distance': 0.20597239809575868,
 'KMPL Vehicle ODO': 0.19838206687868876,
 'Total hours': 18.516666666666666,
 'Total running hours': 15.333333333333334,
 'Total idling hours': 1.8933333333333333,
 'Halt time': 3678.773333333333,
 'Running percentage': 82.80828082808281,
 'Idling percentage': 1022.5022502250224,
 'Average Speed': 34.98145814581408}